In [8]:
from transformers import AutoImageProcessor

checkpoint = "my_awesome_model/checkpoint-13948"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [9]:
from torchvision.transforms import Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
# _transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])
_transforms = Compose([ToTensor(), normalize])

In [10]:
from data_stuff import CSVDataset, Transforms

tran = Transforms(
    image_processor,
    img_size=224,
)

ds_train  = CSVDataset('/mnt/e/data/dataset_train.csv', tran)
ds_val  = CSVDataset('/mnt/e/data/dataset_val.csv', tran)
ds_test  = CSVDataset('/mnt/e/data/dataset_test.csv', tran)

In [11]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

# labels = food["train"].features["label"].names
# label2id, id2label = dict(), dict()
# for i, label in enumerate(labels):
#     label2id[label] = str(i)
#     id2label[str(i)] = label
 
model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=2,
)

In [12]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)



In [13]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    output_dir="my_awesome_model",
    remove_unused_columns=False,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=30,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    processing_class=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

KeyboardInterrupt: 

In [13]:
from datasets import Dataset
samples_df = ds_test.df
test_ds_hf = Dataset.from_pandas(samples_df)
print(test_ds_hf)

Dataset({
    features: ['image', 'label', 'video', 'split'],
    num_rows: 8844
})


In [14]:
from evaluate import evaluator
from transformers import pipeline

task_evaluator = evaluator("image-classification")
pipe = pipeline("image-classification", model=model, image_processor=image_processor)
eval_results = task_evaluator.compute(
    model_or_pipeline=pipe,
    data=test_ds_hf,
    label_mapping={"LABEL_0": 0, "LABEL_1": 1}
)

print(eval_results)

Device set to use cuda:0


{'accuracy': 0.9668701944821347, 'total_time_in_seconds': 114.17548370900113, 'samples_per_second': 77.45971125062792, 'latency_in_seconds': 0.012909937099615687}
